In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer   # Term Frequency - Inverse Document Frequency
import spacy
import tensorflow as tf
nlp = spacy.load('en_core_web_sm')

data = pd.read_csv(
    "spam_ham_dataset.csv",
    #on_bad_lines=False,
    engine="python"     # ParserError: Error tokenizing data. C error: EOF inside string starting at row 1989
)

df = pd.DataFrame(data)
df.dropna()

X = df["text"]
y = df["label_num"]

X

0       Subject: enron methanol ; meter # : 988291\r\n...
1       Subject: hpl nom for january 9 , 2001\r\n( see...
2       Subject: neon retreat\r\nho ho ho , we ' re ar...
3       Subject: photoshop , windows , office . cheap ...
4       Subject: re : indian springs\r\nthis deal is t...
                              ...                        
5166    Subject: put the 10 on the ft\r\nthe transport...
5167    Subject: 3 / 4 / 2000 and following noms\r\nhp...
5168    Subject: calpine daily gas nomination\r\n>\r\n...
5169    Subject: industrial worksheets for august 2000...
5170    Subject: important online banking alert\r\ndea...
Name: text, Length: 5171, dtype: object

### Data Preprocessing

In [2]:
# 1. Lower
X = X.str.lower()

X

0       subject: enron methanol ; meter # : 988291\r\n...
1       subject: hpl nom for january 9 , 2001\r\n( see...
2       subject: neon retreat\r\nho ho ho , we ' re ar...
3       subject: photoshop , windows , office . cheap ...
4       subject: re : indian springs\r\nthis deal is t...
                              ...                        
5166    subject: put the 10 on the ft\r\nthe transport...
5167    subject: 3 / 4 / 2000 and following noms\r\nhp...
5168    subject: calpine daily gas nomination\r\n>\r\n...
5169    subject: industrial worksheets for august 2000...
5170    subject: important online banking alert\r\ndea...
Name: text, Length: 5171, dtype: object

In [3]:
# 2. Remove special characters
spl_chars_removed = []
for sentence in X:
  spl_chars_removed.append(re.sub('[^A-Za-z0-9+]', " ", sentence))
X = spl_chars_removed

X[0]


'subject  enron methanol   meter     988291  this is a follow up to the note i gave you on monday   4   3   00   preliminary  flow data provided by daren      please override pop   s daily volume   presently zero   to reflect daily  activity you can obtain from gas control    this change is needed asap for economics purposes  '

In [4]:
# 4. Lemmatize with Remove stopwords
import spacy
nlp = spacy.load('en_core_web_sm')

tokenized_emails = []
for text in X:
  doc = nlp(text)   # for each email
  tokens = [token.lemma_ for token in doc if not (token.is_stop or token.is_space)]

  tokenized_emails.append(" ".join(tokens))

X = tokenized_emails


In [5]:
nlp2 = spacy.blank('en')
doc2 = nlp2('I am fred eat\'s')

vocab = [token.text for token in nlp('I am fred let"s')]
vocab

['I', 'am', 'fred', 'let"s']

TF-IDF vectorizer takes the vocabulary into account and calculates the Word Vector

In [10]:
# 5. Vectorize
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)    # vectorizer.transform(x) can be used for already fitted Vectorizer


X_tfidf_array = X_tfidf.toarray()
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf_array, y, test_size=0.1, random_state=42)
X_train, X_val , y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [11]:
X_train.shape, y_train.shape         # (5171, 46161)  - where 46161 is the size of the vocabulary

((4187, 45992), (4187,))

In [12]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(45992,)),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)


history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

Epoch 1/10
131/131 [==============================] - 6s 36ms/step - loss: 0.4020 - accuracy: 0.8264 - val_loss: 0.2053 - val_accuracy: 0.9657
Epoch 2/10
131/131 [==============================] - 4s 29ms/step - loss: 0.1282 - accuracy: 0.9890 - val_loss: 0.1026 - val_accuracy: 0.9893
Epoch 3/10
131/131 [==============================] - 4s 31ms/step - loss: 0.0546 - accuracy: 0.9976 - val_loss: 0.0686 - val_accuracy: 0.9914
Epoch 4/10
131/131 [==============================] - 4s 30ms/step - loss: 0.0293 - accuracy: 0.9986 - val_loss: 0.0535 - val_accuracy: 0.9914
Epoch 5/10
131/131 [==============================] - 4s 29ms/step - loss: 0.0179 - accuracy: 0.9998 - val_loss: 0.0450 - val_accuracy: 0.9914
Epoch 6/10
131/131 [==============================] - 4s 29ms/step - loss: 0.0119 - accuracy: 0.9998 - val_loss: 0.0400 - val_accuracy: 0.9914
Epoch 7/10
131/131 [==============================] - 4s 30ms/step - loss: 0.0084 - accuracy: 0.9998 - val_loss: 0.0364 - val_accuracy: 0.9914

In [13]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

17/17 [==============================] - 0s 4ms/step - loss: 0.0470 - accuracy: 0.9846


### Model prediction function

In [14]:
def model_predictions(model, X_predict):
  y_preds = model.predict(X_predict).reshape((X_predict.shape[0], ))  # the rows - number of emails
  preds_spam_ham = ['spam' if prob==1 else 'ham' for prob in np.round(y_preds)]

  return preds_spam_ham


test_emails = vectorizer.transform(
  ['Dinner tonight in my house', 'free money lottery coupons now', 'youve won the lottery click the link', 'Breakfast today in John\'s house', 'The principal asked us to leave']
).toarray()

model_predictions(model, test_emails)

1/1 [==============================] - 0s 111ms/step


['ham', 'spam', 'spam', 'ham', 'ham']

In [15]:
predictions = model_predictions(model, X_test)
print(predictions)

17/17 [==============================] - 0s 5ms/step
['ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'spam', 'spam', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam', 'spam', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'spam', 'spam', 'ham', 'ham', 'ham', 'ham', 'spam', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'h